In [1]:
!pip install -q ultralytics
!pip install -q -U ipywidgets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [2]:
# 2. Импорты
from ultralytics import YOLO
import torch
import os
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import gc
import sys
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Проверяем структуру распакованных данных
DATASET_PATH = '....'  # путь к распакованным данным
test_path = os.path.join(DATASET_PATH, 'test/images')  # путь к тестовым изображениям
print("Проверка данных:")
print(f"Структура директории:\n{os.listdir(DATASET_PATH)}")

Проверка данных:
Структура директории:
['val', 'test', 'train']


In [4]:
# Создаем yaml файл для YOLO
yaml_config = {
    'path': DATASET_PATH,  # сохраненная директория проекта
    'train': 'train/images',  # путь к тренировочным изображениям
    'val': 'val/images',      # путь к валидационным изображениям
    'test': 'test/images',    # путь к тестовым изображениям
    
    'nc': 15,  # количество классов
    'names': [  # имена классов (от 0 до 14)
        'PET (transparent) (green)',
        'PET (transparent) (brown)',
        'PET (transparent) (blue)',
        'PET (transparent)',
        'PET (transparent) (dark blue)',
        'PET (black)',
        'PET (white)',
        'PET (sticker)',
        'PET (flacon)',
        'PET (household chemicals)',
        'PND (household chemicals)',
        'PND packet',
        'Other plastic',
        'Other plastic (transparent)',
        'Not plastic'
    ]
}

# Сохраняем конфигурацию в yaml файл
yaml_path = '' # укажите путь сохранения
with open(yaml_path, 'w') as f:
        yaml.dump(yaml_config, f, default_flow_style=False, sort_keys=False)

In [5]:
model = YOLO('yolo11n.pt')
results = model.train(
    data=yaml_path,
    # Основные параметры
    epochs=100,
    imgsz=832,
    batch=-1,
    device='0' if torch.cuda.is_available() else 'cpu',# Веса для проблемных классов
    
    # Параметры аугментации
    augment=True,
    degrees=10.0,           # поворот на ±10 градусов
    translate=0.2,          # сдвиг на ±10%
    scale=0.5,             # масштабирование на +50%
    shear=2.0,             # сдвиг
    perspective=0.0015,     # перспективные преобразования
    flipud=0.5,            # вертикальное отражение
    fliplr=0.5,            # горизонтальное отражение
    mosaic=1.0,            # мозаичная аугментация
    mixup=0.1,             # смешивание изображений
    copy_paste=0.1,        # копирование и вставка объектов
    hsv_h=0.015,           # изменение оттенка
    hsv_s=0.7,             # изменение насыщенности
    hsv_v=0.4,             # изменение яркости

    
    
    # Параметры оптимизации
    optimizer='AdamW',      # использование AdamW оптимизатора
    
    # Параметры сохранения и валидации
    save=True,            # сохранять результаты
    save_period=10,       # сохранять каждые 10 эпох
    patience=20,          # остановка если нет улучшений после 20 эпох
    plots=True,           # создавать графики 
    project='.....',      # путь для сохранения результатов модели
    name='mod1',
    exist_ok=True,
    
    # Дополнительные параметры
    verbose=True,
    workers=8,
    close_mosaic=10,       # отключить мозаику за 10 эпох до конца
    
    # Параметры валидации
    val=True,
)

print("\nОбучение завершено!")
print(f"Результаты сохранены в: {os.path.join('/kaggle/working/runs', 'exp_extended')}")

100%|██████████| 5.35M/5.35M [00:00<00:00, 96.9MB/s]


Ultralytics 8.3.40 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/waste.yaml, epochs=100, time=None, patience=20, batch=-1, imgsz=832, save=True, save_period=10, cache=False, device=0, workers=8, project=/kaggle/working/runs, name=mod1, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 755k/755k [00:00<00:00, 25.0MB/s]


Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /kaggle/input/d/v0olker/renue-data/data_zip/renue-data/train/labels... 7364 images, 97 backgrounds, 0 corrupt: 100%|██████████| 7364/7364 [00:29<00:00, 249.68it/s]


...

     94/100      9.45G     0.7187     0.7047      1.005         14        832: 100%|██████████| 185/185 [02:32<00:00,  1.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:19<00:00,  1.23it/s]

                   all       1841       7141      0.783      0.805      0.835      0.702


EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 74, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

94 epochs completed in 4.948 hours.
Optimizer stripped from /kaggle/working/runs/mod1/weights/last.pt, 5.5MB
Optimizer stripped from /kaggle/working/runs/mod1/weights/best.pt, 5.5MB

Validating /kaggle/working/runs/mod1/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11n summary (fused): 238 layers, 2,585,077 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:25<00:00,  1.06s/it]


                   all       1841       7141       0.82      0.787      0.847      0.713
PET (transparent) (green)        676        937      0.884      0.902      0.925      0.781
PET (transparent) (brown)        806       1511      0.894      0.891       0.92      0.776
PET (transparent) (blue)        182        193      0.801      0.674      0.808      0.677
     PET (transparent)        571        800      0.844      0.766      0.841      0.689
PET (transparent) (dark blue)        192        201      0.739      0.786      0.859      0.731
           PET (black)        196        199      0.804      0.863      0.877      0.748
           PET (white)        424        508      0.848      0.821      0.886       0.76
         PET (sticker)        225        228      0.844      0.842      0.859      0.714
          PET (flacon)        165        176       0.68      0.636      0.676      0.564
PET (household chemicals)        348        424      0.818      0.762      0.836      0.721
PND